In [1]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
m = MorphAnalyzer()

import re

In [2]:
def normalize_text(text):
    lemmas = []
    for token in simple_word_tokenize(text):
        lemmas.append(m.parse(token)[0].normal_form)
    return ' '.join(lemmas)

Тексты взяты с сайта http://www.vokrugsveta.ru. Ключевые слова к текстам были указаны на сайте под каждой статьей. Всего 4 текста.

In [3]:
texts_names = ['текст 1', 'текст 2', 'текст 3', 'текст 4']
keywords = {}
my_keywords = {}

In [77]:
# Достаем тексты в переменную с текстами - texts, 
# а также мои и оригинальные ключевые слова лемматизируем и кладем в словари keywords и my_keywords соответственно
texts = []
for n in texts_names:
    list_1 = []
    list_2 = []
    text = n + '.txt'
    with open(text, 'r', encoding='utf-8') as t:
        new_text = t.read()
    lines = new_text.split('\n')
    for i in lines[0].split(': ')[1].split(', '):
        list_1.append(normalize_text(i))
    my_keywords[n] = list_1
    for i in lines[1].split(': ')[1].split(', '):
        list_2.append(normalize_text(i))
    keywords[n] = list_2
    texts.append(' '.join(lines[2:]))

In [78]:
# Ключевые слова с сайта
keywords

{'текст 1': ['авантюрный роман',
  'берлин',
  'стен',
  'немецкий граница',
  'гдр',
  'фрг'],
 'текст 2': ['уинстон черчилль',
  'юсуф карша',
  'фотография',
  'фото',
  'история один фотография',
  'портрет',
  'история',
  'человек'],
 'текст 3': ['фирменный блюдо', 'сладость', 'япония', 'вагаси'],
 'текст 4': ['история', 'ограбление', 'живопись', 'искусство']}

In [79]:
# Ключевые слова, размеченные мной
my_keywords

{'текст 1': ['берлинский стен', 'берлин', 'гдр', 'ссср'],
 'текст 2': ['уинстон черчилль', 'черчилль', 'фотография', 'портрет', 'карша'],
 'текст 3': ['десерт', 'сладость', 'япония', 'вагаси'],
 'текст 4': ['картина', 'кража', 'искусство', 'шедевр']}

В целом, мои ключевые слова и ключевые слова с сайта очень похожи.

In [81]:
for i, j in enumerate(texts_names):
    set_1 = set(keywords[j])
    set_2 = set(my_keywords[j])
    print('Точное пересечение моих ключевых слов и ключевых слов с сайта в тексте ' + str(i) + ': ' + str(set_1.intersection(set_2)))

Точное пересечение моих ключевых слов и ключевых слов с сайта в тексте 0: {'берлин', 'гдр'}
Точное пересечение моих ключевых слов и ключевых слов с сайта в тексте 1: {'фотография', 'уинстон черчилль', 'портрет'}
Точное пересечение моих ключевых слов и ключевых слов с сайта в тексте 2: {'сладость', 'япония', 'вагаси'}
Точное пересечение моих ключевых слов и ключевых слов с сайта в тексте 3: {'искусство'}


In [8]:
import RAKE
import nltk

In [9]:
from nltk.corpus import stopwords
stop = stopwords.words('russian')

# Выделим ключевые слова с помощью RAKE и оценим точность, полноту и F-меру.

In [10]:
rake = RAKE.Rake(stop)

In [46]:
rake_keywords = {}
for text, n in zip(texts, texts_names):
    result = rake.run(normalize_text(text), maxWords=2, minFrequency=2)
    new_result = []
    for r in result:
        if r[1] > 1: ## слова с рангом 1 и ниже были совсем дурацкие
            new_result.append(r[0])
    rake_keywords[n] = new_result

In [47]:
rake_keywords
# ну как-то так себе, с третьим текстом вообще никак не справился...
# хотя в 1 и 4 тексте есть штуки, похожие на ключевые слова, но все равно как-то плохо...

{'текст 1': ['бернауэр-штрасса',
  'заграничный паспорт',
  'восточный берлин',
  'территория гдр',
  'берлинский стен',
  'западный берлин',
  'свобода выезд',
  'это',
  'граница',
  'стрелять',
  'выезд',
  'страна',
  'окно',
  'западный',
  'место',
  'вступать',
  'делать',
  'пункт'],
 'текст 2': ['это', 'черчилль'],
 'текст 3': [],
 'текст 4': ['служебный вход',
  'мировой война',
  'след преступник',
  'сработать сигнализация',
  'картина',
  'время',
  'удаться',
  'это',
  'найти',
  'музей',
  'весь',
  'италия',
  'написать',
  'арестовать',
  'похитить']}

In [48]:
def count_prf(kw_true, kw_test):
    prf_dict = {}
    all_F1 = 0
    for n in texts_names:
        c = 0
        for w in kw_true[n]:
            if w in kw_test[n]:
                c += 1
        if c != 0:
            P = c/len(kw_test[n])
            R = c/len(kw_true[n])
            F1 = 2 * ((P*R)/(P+R))
        else:
            P = 0
            R = 0
            F1 = 0
            
        prf_dict[n] = (P, R, F1)
        all_F1 += F1
    mean_F1 = all_F1/4
    return prf_dict, mean_F1    

Применила функцию к своим ключевым словам, потому что с оригинальными во всех текстах точность и полнота были по нулям.

In [41]:
rake_prf, rake_mean_F1 = count_prf(my_keywords, rake_keywords) 

In [86]:
print('Слева направо: номер текста, точность, полнота, F-мера (via rake)')
rake_prf

Слева направо: номер текста, точность, полнота, F-мера (via rake)


{'текст 1': (0.05555555555555555, 0.25, 0.0909090909090909),
 'текст 2': (0, 0, 0),
 'текст 3': (0, 0, 0),
 'текст 4': (0.06666666666666667, 0.25, 0.10526315789473685)}

In [87]:
# хотя так тоже все довольно плохо получилось...
print('Средняя по всем текстам F-мера (via rake): ' + str(rake_mean_F1))

Средняя по всем текстам F-мера (via rake): 0.04904306220095694


# Теперь tf-idf

In [49]:
import numpy as np
from math import log

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [51]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'[A-Za-z]', '', text)
    text = [m.normal_forms(x)[0] for x in text.split() if x not in
            stop]
    return text

In [52]:
corpus = []
for t in texts:
    corpus.append(' '.join(clean_text(t)))

In [53]:
X = vectorizer.fit_transform(corpus)

In [54]:
f_matrix = X.toarray()
tf_matrix = np.transpose(f_matrix)

In [55]:
words = vectorizer.get_feature_names()

In [56]:
N = len(corpus)

all_n = {}
for word in words:
    w_idx = words.index(word)
    all_n[word] = np.count_nonzero(tf_matrix[w_idx])

In [57]:
def create_tf_idf_matrix(tf_matrix, N, all_n, words):
    tf_idf_matrix = []
    for idx, doc in enumerate(corpus):
        tf_idf_matrix.append([])
        for word in words:
            tf_idf = 0
            if word in doc:
                w_idx = words.index(word)
                d_idx = corpus.index(doc)
                tf = tf_matrix[w_idx][d_idx]/len(doc)
                n = all_n[word]
                idf = log(N/n)
                tf_idf = tf * idf
            tf_idf_matrix[idx].append(tf_idf)
    return np.array(tf_idf_matrix)

In [58]:
tf_idf_matrix = create_tf_idf_matrix(tf_matrix, N, all_n, words)

In [59]:
tf_idf_matrix

array([[0.00000000e+00, 2.73161450e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        6.91073959e-04, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 9.25430148e-04],
       [1.54375764e-04, 7.71878820e-05, 3.08751528e-04, ...,
        0.00000000e+00, 1.54375764e-04, 0.00000000e+00]])

In [60]:
tf_idf_matrix.max()

0.006910739586838936

In [67]:
tf_idf_matrix[0].max()

0.002185291603735788

In [72]:
tf_idf_matrix[0]

array([0.        , 0.00027316, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [82]:
tf_idf_keywords = {}
for name, arr in zip(texts_names, tf_idf_matrix):
    result = []
    for word, score in zip(words, arr):
        if score > 0.0015:
            result.append(word)
    tf_idf_keywords[name] = result

In [83]:
## Как будто бы получилось даже лучше, чем с RAKE, несмотря на то, что tf-idf не выделяет словосочетания
tf_idf_keywords

{'текст 1': ['берлин', 'гдр', 'западный', 'стен'],
 'текст 2': ['великобритания',
  'кадр',
  'карша',
  'премьера',
  'премьерминистр',
  'сигара',
  'снимка',
  'фотограф',
  'фотография',
  'черчилль'],
 'текст 3': ['десерт',
  'использовать',
  'колобок',
  'маттить',
  'паста',
  'рисовый',
  'фасоль',
  'церемония',
  'чай',
  'чайный',
  'японский'],
 'текст 4': ['картина', 'кража', 'музей', 'похитить']}

Теперь посчитаем точность, полноту и Ф-меру

In [97]:
tf_idf_prf, tf_idf_mean_F1 = count_prf(keywords, tf_idf_keywords) 

In [98]:
print('Слева направо: номер текста, точность, полнота, F-мера (via tf_idf)')
tf_idf_prf

Слева направо: номер текста, точность, полнота, F-мера (via tf_idf)


{'текст 1': (0.75, 0.5, 0.6),
 'текст 2': (0.1, 0.125, 0.11111111111111112),
 'текст 3': (0, 0, 0),
 'текст 4': (0, 0, 0)}

In [99]:
print('Средняя по всем текстам F-мера (via tf_idf): ' + str(tf_idf_mean_F1))

Средняя по всем текстам F-мера (via tf_idf): 0.17777777777777778


In [102]:
tf_idf_prf, tf_idf_mean_F1 = count_prf(my_keywords, tf_idf_keywords) 

In [105]:
print('Слева направо: номер текста, точность, полнота, F-мера (via tf_idf в сравнении с моими ключевыми словами)')
tf_idf_prf

Слева направо: номер текста, точность, полнота, F-мера (via tf_idf в сравнении с моими ключевыми словами)


{'текст 1': (0.5, 0.5, 0.5),
 'текст 2': (0.3, 0.6, 0.4),
 'текст 3': (0.09090909090909091, 0.25, 0.13333333333333333),
 'текст 4': (0.5, 0.5, 0.5)}

In [106]:
print('Средняя по всем текстам F-мера (via tf_idf в сравнении с моими ключевыми словами): ' + str(tf_idf_mean_F1))

Средняя по всем текстам F-мера (via tf_idf в сравнении с моими ключевыми словами): 0.38333333333333336


С моими ключевыми словами получается как-то поближе, чем с оригинальными...

# Теперь попробуем TextRank с библиотекой summa

In [111]:
from summa import keywords as kw

In [130]:
summa_keywords = {}
for text, n in zip(texts, texts_names):
    result = kw.keywords(normalize_text(text), additional_stopwords=stop, scores=True)
    new_result = []
    for r in result:
        if r[1] > 0.14: 
            new_result.append(r[0])
    summa_keywords[n] = new_result

In [131]:
summa_keywords

{'текст 1': ['год', 'гдр', 'человек', 'шабовск', 'стен', 'это', 'граница'],
 'текст 2': ['черчилль фотограф',
  'премьер',
  'портрет',
  'сигара',
  'юсуф карша'],
 'текст 3': ['десерт', 'японский сладость', 'чайный'],
 'текст 4': ['год', 'самый знаменитый картина', 'полиция', 'однако']}

Как-то странно он выделил ключевые слова, вроде близко к правде, но очень криво... В четвертом тексте вообще получилось плохо

In [132]:
summa_prf, summa_mean_F1 = count_prf(keywords, summa_keywords) 

In [133]:
print('Слева направо: номер текста, точность, полнота, F-мера (via summa)')
summa_prf

Слева направо: номер текста, точность, полнота, F-мера (via summa)


{'текст 1': (0.2857142857142857, 0.3333333333333333, 0.30769230769230765),
 'текст 2': (0.4, 0.25, 0.3076923076923077),
 'текст 3': (0, 0, 0),
 'текст 4': (0, 0, 0)}

In [134]:
print('Средняя по всем текстам F-мера (via summa): ' + str(summa_mean_F1))

Средняя по всем текстам F-мера (via summa): 0.15384615384615385


In [135]:
summa_prf, summa_mean_F1 = count_prf(my_keywords, summa_keywords) 

In [136]:
print('Слева направо: номер текста, точность, полнота, F-мера (via summa в сравнении с моими ключевыми словами)')
summa_prf

Слева направо: номер текста, точность, полнота, F-мера (via summa в сравнении с моими ключевыми словами)


{'текст 1': (0.14285714285714285, 0.25, 0.18181818181818182),
 'текст 2': (0.2, 0.2, 0.20000000000000004),
 'текст 3': (0.3333333333333333, 0.25, 0.28571428571428575),
 'текст 4': (0, 0, 0)}

In [137]:
print('Средняя по всем текстам F-мера (via summa в сравнении с моими ключевыми словами): ' + str(summa_mean_F1))

Средняя по всем текстам F-мера (via summa в сравнении с моими ключевыми словами): 0.1668831168831169


Здесь с моими ключевыми словами тоже получилось лучше, чем с оригинальными, хоть и не слишком

Еще раз посмотрим на средние F-мер по всем 4-м текстам (в сравнении тестовых кл. слов с выделенными мной ключевыми словами)

In [139]:
print('Средняя по всем текстам F-мера (via RAKE в сравнении с моими ключевыми словами): ' + str(rake_mean_F1))
print('Средняя по всем текстам F-мера (via tf-idf в сравнении с моими ключевыми словами): ' + str(tf_idf_mean_F1))
print('Средняя по всем текстам F-мера (via summa в сравнении с моими ключевыми словами): ' + str(summa_mean_F1))

Средняя по всем текстам F-мера (via RAKE в сравнении с моими ключевыми словами): 0.04904306220095694
Средняя по всем текстам F-мера (via tf-idf в сравнении с моими ключевыми словами): 0.38333333333333336
Средняя по всем текстам F-мера (via summa в сравнении с моими ключевыми словами): 0.1668831168831169


Удивительно, но лучше всего справился примитивный tf-idf (несмотря на то, что он выделяет только по одному слову, а словосочетания не выделяет), а rake справился совсем ужасно (практически никак).
Хотя в целом цифры все равно довольно печальные.

Есть проблемы с лемматизацией, особенно имен собственных, из-за чего, как минимум, textRank'у засчиталась ошибка к точности.
Rake выделяет слишком много всего в одних текстах, и наоборот - почти ничего в других. Не очень понятно, почему так происходит...
Tf-idf в принципе выделяет ключевые слова неплохо, но также есть проблемы с лемматизацией в некоторых местах (премьер, премьер-министр). По смыслу выделенные tf-idf'ом слова в большинстве случаев вполне соответствуют истине, но все же их многовато, т.к. все-таки он работает через частоты слов + оценивает их встречаемость в других текстах, а корпус у нас маленький и тексты довольно разные. М.б. на бОльшем корпусе он сработал бы лучше.
TextRank с виду, вроде бы, тоже выделяет не так уж и ужасно, в принципе соответствуя действительности (кроме, разве что, 4 текста). Но местами есть проблемы с лемматизацией - "юсуф карша", а также непонятные словосочетания - "черчилль фотограф".